In [1]:
# connect
import psycopg2
import json
import numpy as np

# more connectstr
# "dbname='sqlbook' user='postgres' host='localhost' password='xxxx'"

inputfile = open('connectstr', 'r')

connstr = None

for i in inputfile:
    connstr = json.loads(i.strip())
    
inputfile.close()

conn = psycopg2.connect(connstr)    
conn.set_session(autocommit = True)
cur = conn.cursor();

In [2]:
def get_cust_item_list(cursor):
    freq = dict()
    items = []
    
    sql = """SELECT c.customerid, pr.productid
          FROM customers c, orders o, orderlines ol, products pr
          WHERE o.customerid = c.customerid
          and ol.orderid = o.orderid
          and pr.productid = ol.productid
          order by c.customerid"""

    cursor.execute(sql)
    
    for i in cursor.fetchall():        
        if i[0] in freq:
            freq[i[0]].append(i[1])
        else:
            freq.update({i[0]:[i[1]]}) # item list per customer
        
        freq[i[0]] = list(set(freq[i[0]]))
        items.append(i[1]) # item list for all customers
    
    return freq, list(set(items))

In [3]:
def symmetric_matrix(freq, items):
    n = len(items)
    m = np.zeros((n,n)) # init matrix
    
    for c in freq: # go through each customer
        item = freq[c] # get the items for each customer
        if len(item) < 2: # skip single item list
            continue
            
        # make permutation of item list pairs
        pairs = [(item[i],item[j]) for i in range(len(item)) for j in range(i+1, len(item))]
        #print 'pairs', pairs
        for p in pairs: # get each pair
            if p[0] == p[1]: # pair cannot be equal
                continue
            
            i = items.index(p[0]) # get index
            j = items.index(p[1])
            
            m[i][j] += 1 # update matrix
            m[j][i] += 1
            
    return m

In [4]:
freq, items = get_cust_item_list(cur)
# freq = dictionary, items bought per customer, key = customerid, value = list of items purchased
# items = list, unique items bought by all customers

# create matrix
matrix = symmetric_matrix(freq, items)
#print items
#print matrix
print matrix.shape

(3990, 3990)


In [5]:
# get sample for double checking
for c in freq:
    item = freq[c]
    if len(item) < 2:
        continue
        
    print item
    i = items.index(item[0])
    j = items.index(item[1])
    print matrix[i][j]
    k = items.index(item[2])
    print matrix[j][k]
    break

[10834, 12820, 12479]
196.0
391.0


In [6]:
# double checking
def check_items(cursor,items):
    print items
    sql = """SELECT c.customerid, pr.productid
          FROM customers c, orders o, orderlines ol, products pr
          WHERE o.customerid = c.customerid
          and ol.orderid = o.orderid
          and pr.productid = ol.productid
          and (pr.productid = {0}
          or pr.productid = {1})
          """.format(items[0],items[1])
    print sql
    cursor.execute(sql)
    users = dict()
    
    for i in cursor.fetchall():
        if i[1] not in items:
            continue
            
        if i[0] not in users:
            users.update({i[0]:[i[1]]})
        else:
            users[i[0]].append(i[1])
        
    count = 0
    for i in users:
        if len(list(set(users[i]))) < 2:
            continue
        
        u = list(set(users[i]))
        print 'u',u
        count += 1
        
    print count

In [8]:
#check_items(cur,[10834,12820])
#check_items(cur,[12820,12479])